# Math genealogy project

MIT License

Copyright (c) 2023
Authors: Venkata Sai Narayana Bavisetty, Karthik Vasu

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In this notebook, we outline the process of downloading Math genealogy data and coordinates for different universities. We will conclude by analyzing the data and providing explanations for some of our findings.


# Downloading math genealogy data

In this notebook, we outline the process of downloading Math genealogy data and coordinates for different universities. We will conclude by analyzing the data and providing explanations for some of our findings.

We utilized the [MGP API](https://mathgenealogy.org:8000/api/v2/MGP/) for data retrieval. Below is the script code for downloading the data. While the script is primarily sourced from the [MGP website](https://mathgenealogy.org:8000/login), we made modifications to implement exponential backoff in case of request failures. This adaptation is crucial for successfully downloading the entire dataset in one go.



We download data in batches of $100$ IDs. Although the website indicates that we can download $10,000$ IDs at once, we encountered difficulties when attempting to do so in batches of $10,000$ or even $1,000$.

In [ ]:
#pip install backoff

In [1]:
import requests
import json
import sys
#import backoff
from getpass import getpass

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov 24 18:57:07 2023

@author: venka
"""



PROTOCOL = "https"
HOSTNAME = "mathgenealogy.org"
PORT = "8000"

# Function to prompt on the console for the user's
# email address and password to use to log in to
# the MGP API. Password is not shown on console
# while the user types it.
def getlogin():
    print("Enter email used for MGP authentication:",end=" ",file=sys.stderr)
    #email = input()
    #password = getpass()
    return {'email': "vb8@illinois.edu", 'password': "2FAeinaitrelo!"}

# Function to log in to the MGP API and get a JWT for authentication.
# authdata is a dict with key email set to the user's email address
# and key password set to the user's password.
# If login is successful, returns a JSON object with key token set
# to the JWT.
# If login is unsuccessful, raises RuntimeError.
def login(authdata):
    r = requests.post(f"{PROTOCOL}://{HOSTNAME}:{PORT}/login", authdata)
    if r.ok:
        r.close()
        return r.json()
    else:
        r.close()
        raise RuntimeError("Failed to authenticate")

# Function to do a query against the MGP API. Returns a string with
# the query result if the query was successfully executed. Raises
# RuntimeError if there is an error.

# endpoint is a string (beginning with /) such as "/api/v2/MGP/acad"
# referring to an API endpoint

# token is a dict with key token containing a JWT.
# The return value from login() is the best way to get this.

# params is a dict structured to contain the GET parameters.
# For example, if quering /api/v2/MGP/acad, params could be
# {'id': '1969'}.
@backoff.on_exception(backoff.expo, (requests.exceptions.Timeout,requests.exceptions.RequestException), max_tries= 8)
def doquery(endpoint,token, params):
    headers = {'x-access-token': token['token']}
    r = requests.get(f"{PROTOCOL}://{HOSTNAME}:{PORT}{endpoint}",headers = headers, params = params)
    if r.ok:
        r.close()
        return r.text



# User modifications can be made here in this portion of the script.
# Alternatively, import the functions above into your own python
# script and write code similar to what is given here as the example.
if __name__ == '__main__':



    # Example for results coming back as JSON

    #for id in range(31):
    for id in range(3100):
        authdata = getlogin()
        token = login(authdata)


        querydata = {'start': id*100, "stop":(id+1)*100}
        endpoint = '/api/v2/MGP/acad/range'
        data = json.loads(doquery(endpoint,token,querydata))
        json_file = "try.json"

        # Save data to a JSON file
        with open(json_file, 'a', encoding='utf-8') as jsonfile:
            json.dump(data, jsonfile, indent=4, ensure_ascii=False)

        print(f"Data has been saved to '{json_file}'.")





In [3]:
import pandas as pd

# Read the JSON file
with open('C:\\Users\\karth\\Code\\MGP\\data\\MGP_data.json', 'r') as file:
    all_data = json.load(file)


In [18]:
data=[]

school_split_len = []
for temp_row in all_data:
    row = temp_row['MGP_academic']

    try:
        school = row['student_data']['degrees'][0]['schools'][0]
    except:
        school = 'None'

    split = school.split(',')
    school_split_len.append(len(split))
    if len(split)>1:
        country = split[-1]
    school = split[0]

    try:
        year = row['student_data']['degrees'][0]['degree_year']
    except:
        year = 'None'
    
    try:
        subject = int(row['student_data']['degrees'][0]['degree_msc'])
    except:
        subject = 'None'

    new_row = {
        'ID':row['ID'],
        'family_name':row['family_name'],
        'given_name':row['given_name'],
        'school':school,
        'year':year,
        'descendants':row['student_data']['descendants']['descendant_count'],
        'country':country,
        'subject':subject
        }
    data.append(new_row)

data = pd.DataFrame(data)

# Downloading the coordinate data

In [ ]:
def get_coordinates(university_name,use_Bing=False,bing_api_key=None):

    # Bing API geocoder
    if use_Bing:
        geolocator = Bing(api_key=bing_api_key)

    # Nominatim geocoder
    else:
        geolocator = Nominatim(user_agent="university_locator")

    university_name_no_filler = remove_fillers(university_name)

    try:
        # Use the geocode method to get the location information
        location = geolocator.geocode(university_name_no_filler)

        if location:
            # Extract latitude and longitude
            latitude, longitude = location.latitude, location.longitude
            return [university_name,latitude,longitude]

        else:
            # if we did not get the location return location in Antartica
            return [university_name,-82,135]
            #print(f"Coordinates for {university_name} not found.")

    except Exception as e:
        #print(f"An error occurred: {e}")
        #If there is an error we return None
        return [university_name,None,None]

def remove_fillers(sentence):

    sentence = str(sentence)
    # List of filler words
    filler_words = ['the', 'of', 'and', 'in', 'to', 'a', 'is', 'that', 'it', 'with', 'as', 'on', 'for', 'at']

    # Split the sentence into words
    words = sentence.split()

    # Remove filler words
    filtered_words = [word for word in words if word.lower() not in filler_words]

    # Join the remaining words to form the new sentence
    new_sentence = ' '.join(filtered_words)

    return new_sentence

In [ ]:
import csv
from tqdm import tqdm

# Set download_coordinates to True if you want to download the coordinates
download_coordinates = False

# If you want to use Bing maps API, set use_Bing to True and enter your API key
use_Bing = False
bing_api_key = None

if download_coordinates:

    import time

    # Using Bing maps API for better performance. This requires a free account and an API key
    from geopy.geocoders import Bing

    # Alternatively we can use OpenStreetMaps API which is free and open source, but does not perform as well as Bing
    from geopy.geocoders import Nominatim

    # Get the unique schools
    schools = data['school'].unique()

    # Loop through the schools and save the recieved coordinates in coord
    coords =[]
    count = 0
    for school in tqdm(schools):
        coord = get_coordinates(school,use_Bing=use_Bing,bing_api_key=bing_api_key)
        coords.append(coord)
        count+=1

        # Save coordinates every 500 iterations
        if count%500==0:
            with open('data/coords'+str(count)+'.csv','w', newline='') as file:
                csv_writer = csv.writer(file)
                csv_writer.writerows(coords)

        # To prevent the API from blocking us
        time.sleep(0.2)

    # Save all the coordinates
    with open('data/coordinates.csv','w', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['school','lat','lon'])
        csv_writer.writerows(coords)


    print("Done Downloading Coordinates")

# Making heatmaps